# Testing emcee for fitting parameters

In [21]:
import numpy as np
from scipy.interpolate import griddata
import emcee

### Simulation Profile

In [22]:
# Select image to analyze
# i = 7, 9, 10, 29, 30 are interesting ones
i = 31

In [23]:
img = (np.load("../profiles/dens_test.npy"))[i]
ny, nx = img.shape

# Griglia cartesiana
x = np.linspace(-3, 3, nx)
y = np.linspace(-3, 3, ny)
xx, yy = np.meshgrid(x, y)

# Griglia polare
r_i = np.linspace(0.1, 3, 300)
theta_i = np.linspace(-np.pi, np.pi, 360)
rr, tt = np.meshgrid(r_i, theta_i)

# Converti griglia polare in x, y
xi = rr * np.cos(tt)
yi = rr * np.sin(tt)

# Prendo i valori dell'immagine
points = np.vstack((xx.flatten(), yy.flatten())).T
values = img.flatten()

# Interpoloazione dell'immagin su una griglia polare
grid_intensity = griddata(points, values, (xi, yi), fill_value=0)

# Per il profilo radiale, prendo la media lungo θ
simulation_profile = np.mean(grid_intensity, axis=0)
simulation_error = np.std(grid_intensity, axis=0)
simulation_error[simulation_error < 1e-8] = 1e-8

### Synthetic Profile

In [24]:
# Emcee
r = np.linspace(0.1, 3, 300)


def model(r, I0, gamma, r_g, sigma, f):
    c = 0.3
    R_cutL = 0.4  # Fixed
    R_cutR = 3.0  # Fixed

    r_sinc = c * (np.pi * ((r - r_g) / sigma))**2

    powerlaw = I0 * ((r / r_g)**(-gamma))
    sinc = np.sin(r_sinc) / r_sinc
    gauss = np.exp((-(r - r_g)**4) / (2 * sigma**4))
    L_cut_factor = (r >= R_cutL).astype(float)
    R_cut_factor = (r <= R_cutR).astype(float)

    gap_factor = (1 - f * (sinc * gauss))

    return powerlaw * gap_factor * L_cut_factor * R_cut_factor


def log_prior(parameters):
    I0, gamma, r_g, sigma, f = parameters
    if 0. < I0 < 1. and 0. < gamma < 1. and 0.9 < r_g < 1.1 and 0.01 < sigma < 0.5 and 0.0 < f < 1.0:
        return 0.0
    return -np.inf


def log_likelihood(parameters, r, data, error):
    model_profile = model(r, *parameters)
    return - 0.5 * np.sum(((data - model_profile) / error)**2)


def log_probability(parameters, r, data, error):
    lp = log_prior(parameters)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(parameters, r, data, error)


def compute_fit_score(r, data, error, theta_best):
    I_model = model(r, *theta_best)
    residuals = (data - I_model) / error
    chi2_red = np.mean(residuals**2)
    score = 1 / (1 + chi2_red)
    return score


# Setup for emcee
initial = np.array([1.0, 1.0, 1.0, 0.1, 0.5])
ndim = len(initial)
nwalkers = 20
initial_position = initial + 1e-3 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(r, simulation_profile, simulation_error))
sampler.run_mcmc(initial_position, 10000, progress=True)
flat_samples = sampler.get_chain(discard=1000, thin=20, flat=True)

theta_best = np.median(sampler.get_chain(discard=1000, flat=True), axis=0)

# Calcolo dello score
score = compute_fit_score(r, simulation_profile, simulation_error, theta_best)

# Best parameters and model
best_params = np.median(flat_samples, axis=0)
best_model = model(r_i, *best_params)


100%|██████████| 10000/10000 [00:13<00:00, 762.36it/s]


In [25]:
# Score calculation
score = compute_fit_score(r, simulation_profile, simulation_error, theta_best)
print(f"Fit score: {score:.3f}")
print("Best parameters:")
for label, value in zip(["I0", "gamma", "r_g", "sigma", "f"], best_params):
    print(f"{label}: {value:.4f}")

Fit score: 0.113
Best parameters:
I0: 0.0007
gamma: 0.3002
r_g: 0.9703
sigma: 0.4991
f: 0.9998
